In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, ImageDataGenerator, load_img

In [23]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
training_set = train_datagen.flow_from_directory(
        "hotdog-nothotdog/train",
        batch_size=32,
        class_mode='binary',
        target_size=(128, 128),
) #class_mode is either binary or categorical

Found 3000 images belonging to 2 classes.


In [25]:
test_set = test_datagen.flow_from_directory(
        "hotdog-nothotdog/test",
        target_size=(128, 128),
        batch_size=32,
        class_mode='binary')

Found 644 images belonging to 2 classes.


In [26]:
cnn = Sequential([
  Conv2D(32, 3, activation='relu', input_shape=(128, 128, 3)),
  MaxPooling2D(2, strides=2),
  Conv2D(64, 3, activation='relu'),
  MaxPooling2D(2, strides=2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(1, activation='sigmoid')

])

In [27]:
cnn.compile(
    optimizer='rmsprop',
    loss='bce',
    metrics=['accuracy']
)

es = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

cnn.fit(
    x= training_set,
    validation_data=test_set,
    batch_size=32,
    epochs=50,
    callbacks=es,
)

Epoch 1/50
94/94 [==============================] - 19s 199ms/step - loss: 1.0523 - accuracy: 0.6043 - val_loss: 0.6178 - val_accuracy: 0.6786
Epoch 2/50
94/94 [==============================] - 18s 187ms/step - loss: 0.6285 - accuracy: 0.6763 - val_loss: 0.6282 - val_accuracy: 0.6506
Epoch 3/50
94/94 [==============================] - 28s 299ms/step - loss: 0.5344 - accuracy: 0.7340 - val_loss: 0.6364 - val_accuracy: 0.6910
Epoch 4/50
94/94 [==============================] - 22s 236ms/step - loss: 0.4515 - accuracy: 0.7913 - val_loss: 0.7528 - val_accuracy: 0.6491
Epoch 5/50
94/94 [==============================] - 21s 222ms/step - loss: 0.3642 - accuracy: 0.8363 - val_loss: 0.7207 - val_accuracy: 0.6832
Epoch 6/50
94/94 [==============================] - 21s 228ms/step - loss: 0.2679 - accuracy: 0.8910 - val_loss: 0.7853 - val_accuracy: 0.6848


In [ ]:
cnn.evaluate(test_set)

In [21]:
test_image = image.load_img('hotdog-nothotdog/test/hotdog/1510.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] > 0.5:
    prediction = 'hotdog'
else:
    prediction = 'nothotdog'
    
prediction

1/1 [==============================] - 0s 14ms/step


'nothotdog'